In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
# Any results you write to the current directory are saved as output.

import os
print(os.listdir("../input"))
# Read the training data
train = pd.read_csv('../input/train.csv')
# Read the testing data
test = pd.read_csv('../input/test.csv')
print(train.head())


# Display meta-data for training and testing data
train.info()
print("_"*50+"\n") # Print line
test.info()
print("\n"*5)
# Display columns (from training) to select what to use for prediction
print(train.columns)

# print(train.Age.median())

# Repalce NAN values with median age
train.Age = train.Age.fillna(train.Age.median())
# Replace NAN with S
train.Embarked.fillna("S", inplace=True)
# Replace string objects with ints
train.Sex = train.Sex.map({"male": 0, "female":1})
train.Embarked = train.Embarked.map({"S":0,"C":1,"Q":2}) 





# What will be predicted
train_y = train.Survived


# What will be used to make prediction
predictor_column = ['Sex','Pclass','Age','Embarked','Fare','Parch']

# Create training predictor data
train_X = train[predictor_column]

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# x_train = sc.fit_transform(x_train)
# x_test = sc.transform(x_test)

train_X = sc.fit_transform(train_X)


# my_model = RandomForestRegressor(max_depth=5, n_estimators=30)
my_model = RandomForestRegressor(max_depth = 6, n_estimators = 300)

# Fiting the model
my_model.fit(train_X, train_y)
#*************
#	
#	Test data 
#
#*************

# Replace NAN value with median age
test.Age = test.Age.fillna(test.Age.median())
test.Fare = test.Fare.fillna(test.Fare.median())
test.Embarked.fillna("S", inplace=True)

test.Sex = test.Sex.map({"male": 0, "female":1})
test.Embarked = test.Embarked.map({"S":0,"C":1,"Q":2}) 

test_X = test[predictor_column]
test_X = sc.fit_transform(test_X)

predicted_survival = my_model.predict(test_X)
predicted_survival_clean = []

print(predicted_survival)


for item in range(len(predicted_survival)):
	if predicted_survival[item] >= 0.5:
		predicted_survival_clean.append(1)
	else:
		predicted_survival_clean.append(0)
print(predicted_survival_clean)


my_submission = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predicted_survival_clean})

my_submission.to_csv('my_submission.csv', index=False)
print(my_submission)


In [ ]:
train.sample(15)

In [ ]:
train.info()

In [ ]:
train.Embarked.unique()
# train.Embarked.fillna("S", inplace=True)
# train.Embarked.unique()

In [ ]:
train.Parch.unique()

In [ ]:
train.Parch.tail

In [ ]:
# train.describe()
train.info()
# train.hist(bins = 25, figsize=(20,15))


In [ ]:
test.Fare

In [ ]:
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train[['Parch', 'Survived']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train[['SibSp', 'Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
corr_matrix = train.corr()
corr_matrix["Survived"].sort_values(ascending=False)



In [ ]:
corr_matrix = train.corr()
corr_matrix[""].sort_values(ascending=False)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
        {},
# 		{'n_estimators': [3,10,30,150,200,250,275,300,325,3000],'max_depth': [1,2,4,5,6,8],},
		# {'bootstrap':[False],'n_estimators':[3,10],'max_features':[2,3,4]},
	]
my_Model = RandomForestRegressor()

grid_search = GridSearchCV(my_Model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(train_X, train_y)
print("The best parameters for RandomForestRegressor are ",grid_search.best_params_)

from sklearn.metrics import mean_squared_error
import numpy as np
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
	print(np.sqrt(-mean_score), params)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
		{'n_estimators': [3,10,20,30,50,60,90,100,150,200,300,500], 'max_depth': [2,6,8,10,12,16,20],},
    # {'bootstrap':[False],'n_estimators':[3,10],'max_features':[2,3,4]},
        
        
	]
my_Model = RandomForestRegressor()
grid_search = GridSearchCV(my_Model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(train_X, train_y)
print("The best parameters for RandomForestRegressor are ",grid_search.best_params_)

from sklearn.metrics import mean_squared_error
import numpy as np
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
	print(1 - np.sqrt(-mean_score), params)